In [1]:
import transformers

In [2]:
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

from transformers import(
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    get_linear_schedule_with_warmup,
    default_data_collator
)

from datasets import(
    Dataset,
    load_dataset
)

import torch
import os
from torch import optim
from torch.utils.data import dataloader
from tqdm import tqdm



In [3]:
from peft import(
    LoraConfig,
    PeftConfig,
    PromptEncoderConfig,
    PrefixTuningConfig,
    IA3Config,
    TaskType,
    get_peft_model,
    PromptTuningInit,
    PromptTuningConfig,
    prepare_model_for_kbit_training

)

In [4]:
device="cuda"

model_path="mistralai/Mistral-7B-v0.1"
lr=1e-3
num_epochs=1
batch_size=1
steps=150
save_location="C:/Users/PRASANNA P/Desktop/AI code/gpt2-finetune"
tokenizer=AutoTokenizer.from_pretrained(model_path)

In [5]:
bnb_8bit= BitsAndBytesConfig(load_in_8bit=True)
bnb_4bit=BitsAndBytesConfig(load_in_4bit=True)
bnb_4bit_nested=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True)

bnb_4bit_nested_nf4=BitsAndBytesConfig(load_in_4bit=True,
                                       bnb_4bit_quant_type='nf4',
                                       bnb_4bit_use_double_quant=True,
                                       bnb_4bit_compute_dtype=torch.bfloat16)

In [7]:
model= AutoModelForCausalLM.from_pretrained(model_path,
                                            quantization_config= bnb_4bit,
                                            device_map="auto")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

C:\Users\PRASANNA P\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PRASANNA P\.cache\huggingface\hub\models--mistralai--Mistral-7B-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/78/c6/78c6450f2e4f8ec5f8c6c5ac2c4ecff0e6c9dadc650fa7daf2d5bc0323dc99f6/9742cb4764964155b7a5f35eefad651f590006091ddeb536863d6c5865cca1b9?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1718432993&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODQzMjk5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83OC9jNi83OGM2NDUwZjJlNGY4ZWM1ZjhjNmM1YWMyYzRlY2ZmMGU2YzlkYWRjNjUwZmE3ZGFmMmQ1YmMwMzIzZGM5OWY2Lzk3NDJjYjQ3NjQ5NjQxNTViN2E1ZjM1ZWVmYWQ2NTFmNTkwMDA2MDkxZGRlYjUzNjg2M2Q2YzU4NjVjY2ExYjk%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=lyzmKl5w2YE9%7E8M1WLhDlqoH042b99RaYPRHlXOWzu8pJlkdBdcYpNGLrS7uIYn8pfhreRsiVmO%7E-QAp8MbYGNvbsIi6NPaLnBdsYf0E4zUxExQX93k6xQji%7EXjAMh1jUya7dEvaecBVPyAo8cVEg0mDVbuBOQO6G4xZgQv4UMLSQKt4tqLT00n8zMmR9Comv2LHF5eIMsUy29zdha

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs.huggingface.co\', port=443): Max retries exceeded with url: /repos/78/c6/78c6450f2e4f8ec5f8c6c5ac2c4ecff0e6c9dadc650fa7daf2d5bc0323dc99f6/9742cb4764964155b7a5f35eefad651f590006091ddeb536863d6c5865cca1b9?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1718432993&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODQzMjk5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83OC9jNi83OGM2NDUwZjJlNGY4ZWM1ZjhjNmM1YWMyYzRlY2ZmMGU2YzlkYWRjNjUwZmE3ZGFmMmQ1YmMwMzIzZGM5OWY2Lzk3NDJjYjQ3NjQ5NjQxNTViN2E1ZjM1ZWVmYWQ2NTFmNTkwMDA2MDkxZGRlYjUzNjg2M2Q2YzU4NjVjY2ExYjk~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=lyzmKl5w2YE9~8M1WLhDlqoH042b99RaYPRHlXOWzu8pJlkdBdcYpNGLrS7uIYn8pfhreRsiVmO~-QAp8MbYGNvbsIi6NPaLnBdsYf0E4zUxExQX93k6xQji~XjAMh1jUya7dEvaecBVPyAo8cVEg